In [1]:
import os

In [2]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [4]:
from pathlib import Path

CONFIG_FILE_PATH = Path("../config/config.yaml")
PARAMS_FILE_PATH = Path("../config/params.yaml")

print(CONFIG_FILE_PATH, CONFIG_FILE_PATH.exists())  # should print True
print(PARAMS_FILE_PATH, PARAMS_FILE_PATH.exists())  # True if params.yaml exists


..\config\config.yaml True
..\config\params.yaml False


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=None   # make it optional
    ):
        self.config = read_yaml(config_filepath)
        if params_filepath and Path(params_filepath).exists():
            self.params = read_yaml(params_filepath)
        else:
            self.params = {}  # empty dict if no params.yaml

        create_directories([self.config['artifacts_root']])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [6]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size



In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
     
    def download_file(self)-> str:
        ''' Fetch data from the url '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-17 09:34:53,311: INFO: common: yaml file: ..\config\config.yaml loaded successfully]
[2025-11-17 09:34:53,311: INFO: common: created directory at: artifacts]
[2025-11-17 09:34:53,318: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-17 09:34:53,318: INFO: 181254727: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=f7e84d16-5ff6-4d42-b720-9a7938e57beb
To: c:\Users\divya\Desktop\End-to-End-chest-cancer-classification-using-MLflow-DVC\research\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:12<00:00, 3.90MB/s]

[2025-11-17 09:35:09,553: INFO: 181254727: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [9]:
import os
print(os.getcwd())


c:\Users\divya\Desktop\End-to-End-chest-cancer-classification-using-MLflow-DVC\research
